In [ ]:
!git clone https://github.com/LINKS-Foundation-CPE/Master-QCC-2022-2023.git

In [ ]:
!pip install pulser==0.7
!pip install networkx==2.8.7
!pip install matplotlib==3.5.1

In [ ]:
%cd /content/Master-QCC-2022-2023/

In [ ]:
import networkx as nx
from src.itMIS.visualize_graph import *
from src.itMIS.utils import *
from src.itMIS.MIS_kernel import PulserMISSolver
from src.basic_MIS.utils import *

## Iterative MIS for graph coloring

In [ ]:
# read graph
file_path="graphs/{}G_{}ud.gpickle".format(7, 2)
G = nx.read_gpickle(file_path)
G = plot_initial_graph_nx(G, 7)
orig_G = G.copy()


In [ ]:
# pulser embedding
rabi_freq, blockade_radius = compute_rydberg(G)
print(f'Blockade Radius {blockade_radius:.2f} μm')
plot_initial_graph_pulser(G, blockade_radius)

## Greedy-itMIS
Solve iteratively MIS (Maximum Independent Set) problem to retrieve a feasible coloring. At each step, a subset of independent vertexes is removed from G and a color is assigned to them.

In [ ]:
coloring =  dict.fromkeys(G.nodes(), -1)
num_colors = 0

# plot initial graph
plot_sol(coloring, orig_G, -1)

while len(G.nodes())> 0:   
    if len(G.edges())>0: 
        # there are still conflict to be solved
        pulser_MIS_solver = PulserMISSolver(G)
        # solve MIS
        solutions = pulser_MIS_solver.solve_Pulser()
        num_sol = len(solutions)     
        print('Found {} solutions'.format(num_sol))       
        for sol in range(num_sol):  
            x = solutions[sol]      
            if is_MIS(x, G):
                # the solution is indipendent and maximal
                H, MIS_set = compute_subgraph(x, G)
                print(f'Found MIS solution at position {sol}')
                break
        G=H
    else:
        # the same color can be assigned to all the remaining nodes 
        MIS_set=G.nodes()
        G = G.subgraph([])
    num_colors+=1
    # update the coloring
    for graph_node in MIS_set:
        coloring[graph_node]=num_colors 
    # plot coloring
    plot_sol(coloring.copy(), orig_G, num_colors)
        
print('Iterative MIS solved with {} colors'.format(num_colors))  

In [ ]:
# compare coloring results with the one obtained with different nx heuristics
NetworkxGC(orig_G)